In [1]:
import pandas as pd
import numpy as np
# fast ai is used only for easy data transformations
from fastai import *
from fastai.tabular import *
from fastai.tabular.all import *
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

In [2]:
train_df=pd.read_csv("../sample_data/train.csv",low_memory=False)
test_df=pd.read_csv("../sample_data/test.csv",low_memory=False)

In [3]:
# Remove rows where all elements are NaN
org_train_len=len(train_df)
train_df = train_df.dropna(how='all')
clean_train_len=len(train_df)
org_test_len=len(test_df)
test_df = test_df.dropna(how='all')
clean_test_len=len(test_df)

In [4]:
train_df['date'] = pd.to_datetime(train_df['date'], errors='coerce')
# Count the number of NaN values in the date column
nan_count = train_df['date'].isna().sum()
len(train_df),nan_count

/var/folders/1m/ps1kwqdn5p1bdj1943362s500000gq/T/ipykernel_87578/1682223351.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_df['date'] = pd.to_datetime(train_df['date'], errors='coerce')


(6480, 0)

In [5]:
# Define a conversion function with NaN handling
def convert_to_liters(value):
    # Check for NaN (or similar) values
    if pd.isna(value):
        # Choose your strategy here: return None, or a default value, or raise an error, etc.
        return np.nan  # Here, we choose to return NaN as is
    # Split the number and unit
    number, unit = float(value[:-2]), value[-2:]
    # Convert to liters if necessary
    if unit == 'ml':
        return number / 1000  # Convert ml to l
    elif unit == 'lt':
        return number
    else:
        raise ValueError(f"Unknown unit: {unit}")

In [6]:
# Apply the conversion to the entire column
train_df['capacity_liters'] = train_df['capacity'].apply(convert_to_liters)
test_df['capacity_liters'] = test_df['capacity'].apply(convert_to_liters)

In [7]:
train_df = train_df.drop(columns=['capacity'])
test_df = test_df.drop(columns=['capacity'])

In [8]:
add_datepart(train_df, 'date')
add_datepart(test_df,'date')

/opt/anaconda3/envs/ML_OPS/lib/python3.10/site-packages/fastai/tabular/core.py:23: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)
/opt/anaconda3/envs/ML_OPS/lib/python3.10/site-packages/fastai/tabular/core.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)


,id,city,lat,long,pop,shop,brand,container,price,quantity,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,6480,Athens,37.97945,23.71622,664046,shop_1,kinder-cola,plastic,3.10,7056,...,31,2,31,True,False,False,False,False,False,1.517357e+09
1,6481,Athens,37.97945,23.71622,664046,shop_1,kinder-cola,can,0.85,12490,...,31,2,31,True,False,False,False,False,False,1.517357e+09
2,6482,Athens,37.97945,23.71622,664046,shop_1,adult-cola,glass,0.83,26640,...,31,2,31,True,False,False,False,False,False,1.517357e+09
3,6483,Athens,37.97945,23.71622,664046,shop_1,orange-power,glass,0.54,41892,...,31,2,31,True,False,False,False,False,False,1.517357e+09
4,6484,Athens,37.97945,23.71622,664046,shop_1,orange-power,plastic,0.83,22923,...,31,2,31,True,False,False,False,False,False,1.517357e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,7555,Athens,37.97945,23.71622,664046,shop_1,kinder-cola,plastic,2.52,13760,...,31,0,365,True,False,True,False,True,False,1.546214e+09
1076,7556,Athens,37.97945,23.71622,664046,shop_1,orange-power,plastic,2.18,16309,...,31,0,365,True,False,True,False,True,False,1.546214e+09
1077,7557,Patra,38.24444,21.73444,168034,shop_6,kinder-cola,can,0.85,24378,...,31,0,365,True,False,True,False,True,False,1.546214e+09
1078,7558,Thessaloniki,40.64361,22.93086,354290,shop_4,adult-cola,plastic,2.17,20691,...,31,0,365,True,False,True,False,True,False,1.546214e+09


In [9]:
train_df['quantity'] = np.log(train_df['quantity'])
test_df['quantity'] = np.log(test_df['quantity'])

In [10]:
cat_names = ['city', 'shop', 'brand', 'container']
cont_names = ['lat', 'long', 'pop','capacity_liters','price']
procs = [Categorify, FillMissing, Normalize]
y_names = 'quantity'

In [11]:
train_df

,id,city,lat,long,pop,shop,brand,container,price,quantity,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0.0,Athens,37.97945,23.71622,672130.0,shop_1,kinder-cola,glass,0.96,9.494014,...,31,1,31,True,False,False,False,False,False,1.327968e+09
1,1.0,Athens,37.97945,23.71622,672130.0,shop_1,kinder-cola,plastic,2.86,8.813885,...,31,1,31,True,False,False,False,False,False,1.327968e+09
2,2.0,Athens,37.97945,23.71622,672130.0,shop_1,kinder-cola,can,0.87,9.195024,...,31,1,31,True,False,False,False,False,False,1.327968e+09
3,3.0,Athens,37.97945,23.71622,672130.0,shop_1,adult-cola,glass,1.00,9.905984,...,31,1,31,True,False,False,False,False,False,1.327968e+09
4,4.0,Athens,37.97945,23.71622,672130.0,shop_1,adult-cola,can,0.39,10.154091,...,31,1,31,True,False,False,False,False,False,1.327968e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6475,6475.0,Athens,37.96245,23.68708,665871.0,shop_3,orange-power,plastic,1.02,10.410335,...,31,6,365,True,False,True,False,True,False,1.514678e+09
6476,6476.0,Larisa,39.63689,22.41761,144302.0,shop_5,orange-power,can,0.47,10.757286,...,31,6,365,True,False,True,False,True,False,1.514678e+09
6477,6477.0,Patra,38.24444,21.73444,168501.0,shop_6,adult-cola,glass,1.02,10.772854,...,31,6,365,True,False,True,False,True,False,1.514678e+09
6478,6478.0,Thessaloniki,40.64361,22.93086,353001.0,shop_4,gazoza,plastic,1.34,10.207842,...,31,6,365,True,False,True,False,True,False,1.514678e+09


In [12]:

tdf_train = TabularPandas(train_df, procs=procs, cat_names=cat_names, cont_names=cont_names,y_block=RegressionBlock(),
                   y_names=y_names)
tdf_test = TabularPandas(test_df, procs=procs, cat_names=cat_names, cont_names=cont_names,y_block=RegressionBlock(),
                   y_names=y_names)

In [13]:
df_train = tdf_train.items
df_test = tdf_test.items

In [14]:
df_test

,id,city,lat,long,pop,shop,brand,container,price,quantity,...,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,lat_na,long_na,capacity_liters_na
0,6480,1,-0.193103,0.416100,1.341481,1,3,3,2.245355,8.861633,...,True,False,False,False,False,False,1.517357e+09,1,1,1
1,6481,1,-0.193103,0.416100,1.341481,1,3,1,-0.468544,9.432684,...,True,False,False,False,False,False,1.517357e+09,1,1,1
2,6482,1,-0.193103,0.416100,1.341481,1,1,2,-0.492668,10.190169,...,True,False,False,False,False,False,1.517357e+09,1,1,1
3,6483,1,-0.193103,0.416100,1.341481,1,5,2,-0.842459,10.642850,...,True,False,False,False,False,False,1.517357e+09,1,1,1
4,6484,1,-0.193103,0.416100,1.341481,1,5,3,-0.492668,10.039896,...,True,False,False,False,False,False,1.517357e+09,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,7555,1,-0.193103,0.416100,1.341481,1,3,3,1.545772,9.529521,...,True,False,True,False,True,False,1.546214e+09,1,1,1
1076,7556,1,-0.193103,0.416100,1.341481,1,5,3,1.135672,9.699472,...,True,False,True,False,True,False,1.546214e+09,1,1,1
1077,7557,4,-0.032382,-1.414598,-0.814148,6,3,1,-0.468544,10.101437,...,True,False,True,False,True,False,1.546214e+09,1,1,1
1078,7558,5,1.422763,-0.309388,-0.004694,4,1,3,1.123610,9.937454,...,True,False,True,False,True,False,1.546214e+09,1,1,1


In [15]:
y_train=df_train['quantity']
X_train = df_train.drop('quantity', axis=1)
y_test=df_test['quantity']
X_test = df_test.drop('quantity', axis=1)

In [16]:
y_train

0        9.494015
1        8.813885
2        9.195024
3        9.905985
4       10.154091
          ...    
6475    10.410336
6476    10.757286
6477    10.772855
6478    10.207843
6479    10.322625
Name: quantity, Length: 6480, dtype: float32

In [17]:
X_train

,id,city,lat,long,pop,shop,brand,container,price,capacity_liters,...,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,lat_na,long_na,capacity_liters_na
0,0.0,1,-0.195568,0.413876,1.361571,1,3,2,-0.281130,-0.535599,...,True,False,False,False,False,False,1.327968e+09,1,1,1
1,1.0,1,-0.195568,0.413876,1.361571,1,3,3,2.051577,1.402810,...,True,False,False,False,False,False,1.327968e+09,1,1,1
2,2.0,1,-0.195568,0.413876,1.361571,1,3,1,-0.391627,-0.865128,...,True,False,False,False,False,False,1.327968e+09,1,1,1
3,3.0,1,-0.195568,0.413876,1.361571,1,1,2,-0.232020,-0.535599,...,True,False,False,False,False,False,1.327968e+09,1,1,1
4,4.0,1,-0.195568,0.413876,1.361571,1,1,1,-0.980942,-0.865128,...,True,False,False,False,False,False,1.327968e+09,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6475,6475.0,1,-0.205915,0.386978,1.334687,3,5,3,-0.207465,1.402810,...,True,False,True,False,True,False,1.514678e+09,1,1,1
6476,6476.0,3,0.813132,-0.784783,-0.905626,5,5,1,-0.882723,-0.865128,...,True,False,True,False,True,False,1.514678e+09,1,1,1
6477,6477.0,4,-0.034299,-1.415371,-0.801683,6,1,2,-0.207465,-0.535599,...,True,False,True,False,True,False,1.514678e+09,1,1,1
6478,6478.0,5,1.425809,-0.311037,-0.009194,4,2,3,0.185412,1.402810,...,True,False,True,False,True,False,1.514678e+09,1,1,1


In [18]:
y_test

0        8.861633
1        9.432684
2       10.190169
3       10.642850
4       10.039896
          ...    
1075     9.529521
1076     9.699472
1077    10.101437
1078     9.937454
1079    10.111112
Name: quantity, Length: 1080, dtype: float32

In [20]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [21]:
models = {
    "                   K-Nearest Neighbors": KNeighborsRegressor(),
    "                     Linear Regression": LinearRegression(),
    "                 Ridge (L2) Regression": Ridge(),
    "Support Vector Machine (Linear Kernel)": LinearSVR(),
    "   Support Vector Machine (RBF Kernel)": SVR(),
    "                         Decision Tree": DecisionTreeRegressor(),
    "                        Neural Network": MLPRegressor(),
    "                         Random Forest": RandomForestRegressor(),
    "                     Gradient Boosting": GradientBoostingRegressor()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

                   K-Nearest Neighbors trained.
                     Linear Regression trained.
                 Ridge (L2) Regression trained.


/opt/anaconda3/envs/ML_OPS/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=5.05576e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/opt/anaconda3/envs/ML_OPS/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/envs/ML_OPS/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                         Decision Tree trained.
                        Neural Network trained.
                         Random Forest trained.
                     Gradient Boosting trained.


In [23]:
for name, model in models.items():
    # Generate predictions
    y_pred = model.predict(X_test)
    
    # Apply log transformation (adding a small constant to avoid log(0))
    y_test_exp = np.exp(y_test)
    y_pred_exp = np.exp(y_pred)
    
    # Calculate R^2 score on the log-transformed values
    score = r2_score(y_test_exp, y_pred_exp)
    
    print(f"{name} R^2: {score:.5f}")

    # Calculate MAE
    mae = mean_absolute_error(y_test_exp, y_pred_exp)
    print(f"{name} MAE : {mae}")

    # Calculate MSE
    mse = mean_squared_error(y_test_exp, y_pred_exp)
    print(f"{name} MSE: {mse}")

    # Calculate RMSE
    rmse = np.sqrt(mse)
    print(f"{name} RMSE: {rmse}")


                   K-Nearest Neighbors R^2: -0.18245
                   K-Nearest Neighbors MAE : 14789.150390625
                   K-Nearest Neighbors MSE: 309367616.0
                   K-Nearest Neighbors RMSE: 17588.849609375
                     Linear Regression R^2: -0.08731
                     Linear Regression MAE : 12203.670490962373
                     Linear Regression MSE: 284476446.1337131
                     Linear Regression RMSE: 16866.42956092703
                 Ridge (L2) Regression R^2: 0.61093
                 Ridge (L2) Regression MAE : 7341.115261195926
                 Ridge (L2) Regression MSE: 101792158.71894543
                 Ridge (L2) Regression RMSE: 10089.21001461192
Support Vector Machine (Linear Kernel) R^2: -3.29471
Support Vector Machine (Linear Kernel) MAE : 29359.875675229672
Support Vector Machine (Linear Kernel) MSE: 1123634379.1165555
Support Vector Machine (Linear Kernel) RMSE: 33520.65600665589
   Support Vector Machine (RBF Kernel) R^2:

In [25]:
y_pred=np.exp(models["                     Gradient Boosting"].predict(X_test))

In [26]:
y_pred

array([ 8004.54967931, 12126.85817561, 20452.28106689, ...,
       25165.23748741, 23733.44728427, 22553.57087214])

In [28]:
np.exp(y_test)

0        7055.998047
1       12490.003906
2       26640.005859
3       41891.988281
4       22923.000000
            ...     
1075    13759.998047
1076    16309.000977
1077    24378.005859
1078    20691.001953
1079    24615.007812
Name: quantity, Length: 1080, dtype: float32